# Hi-Tech Girls Project by Monika Kaczan

The main goal of this projects was to explore relationships between weather and functioning of BTS stations across Poland. I was given the data regarding weather in weather stations, as well as data regarding location and function of BTS stations.

To explore relationships between the weather and functioning of BTS stations, I took the following steps:

1. Data preprocessing
   - Loading and transforming the data regarding the weather
   - Completing the information about weather stations
   - Checking the data correctness
   - Aggregating the data about weather stations
   - Handling missing values
   - Loading the data on BTS stations
   - Assigning BTS stations data on the weather
   - Adding the data on BTS malfunctioning
2. Data analysis
   - Preliminary analysis for the weather stations data
   - Preliminary analysis for the BTS stations data
   - Comparing Orange and other providers
   - Visualization of stations on the map
   - Identification of stations that are exposed to the most extreme weather changes
   - Assesing the impact of weather on BTS malfunctioning
3. Training machine learning models
   - Clustering: 
     - K-Means
   - Classification model predicting a high percentage of anomaly occurences 
     - Logit
     - Ridge Classifier
     - XGBoost
     - K nearest neighbours
4. Conclusions
   - Further project development
   - Applying conclusions drawn from the data in practice





Throught the notebook I am using the following variables which are further explained when neccessary (units assumed from the data):
- A - air temperature (°C)
- B - ground temperature (°C)
- C - average wind speed (m/s)
- D - daily total precipitation (mm)
- E - relative air humidity (%)
- anomalies - according to the definition provided I am refering to the percentage of anomalies in functioning of a BTS station throught the month. 

Throught the analysis I have used several additional files, for example Poland shapefile for vizualization or files with ML models that took long time to calculate.



## Data preprocessing

### Loading and transforming the data regarding weather

Loading the data

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
# Importing libraries

import numpy as np
import pandas as pd

import os
import glob

# Choosing proper data files

path = os. getcwd() + '/data'
csv_files = glob.glob(path + "./[A-E]_*")
df_list = (pd.read_csv(file, encoding = "ISO-8859-2", sep = ";", decimal = ',', low_memory = False) for file in csv_files)

# Combining information from all the files into one dataframe

df_raw = pd.concat(df_list, ignore_index = True)
df_master = df_raw

Exploring the data

In [3]:
# Let's look at the data and change columns names

df_master.info()
display(df_master)
df_master.columns = ["id", "coef", "date", "value"]

Handling missing values

In [4]:
# Let's look at the NAs

df_master.isna().sum()
# We see that we have lot's of NAs

df_master.isnull().any(axis=1).sum()
# However they are just empty rows so we can delete them

df_master = df_master.dropna(how = 'all').reset_index(drop = True)

Handling duplicates

In [5]:
# Let's check the data for duplicates

# Checking duplictes by rows
df_master[df_master.duplicated()]

# Deleting duplicates
df_master = df_master.drop_duplicates()


# Checking if we have rows with the same ID, Date and Coef but different

df_master[df_master.duplicated(subset = ['id','coef', 'date'])]
# We have observations with the same ID, Coef and Date, but different Value. 
# I assume the last (by row number) Value is correct, for example somebody made a mistake and tried to overrite it. 

# Deleting those duplicates
df_master = df_master.drop_duplicates(subset = ['id','coef', 'date'], keep = 'last')

Changing data format

In [6]:
df_master = df_master.astype({'id':'int', 'coef': 'str'})
df_master['date'] = pd.to_datetime(df_master['date'], format = "%d.%m.%Y %H:%M")

Unstacking - we have all parameters in the same column, so we can unstack them and move them to different columns.

In [7]:
# As we have all parameters in the same column, we can unstack them and move them to different columns.

# First, let's check if parameters are written correctly.

df_master['coef'].unique()
# We see that we have Coef 'B' and 'B '. I assume that it is a mistake so I merge them.

df_master['coef'] = df_master['coef'].replace('B ', 'B')

In [8]:
# Unstacking

df_unstacked = df_master.pivot_table(index = ['id', 'date'], columns = ['coef'], values = 'value').reset_index()
df_unstacked

# Checking if everything went okay and we have the same number of observations as before

# df_master['Coef'].value_counts(dropna = True).sort_values()
# df_unstacked.count().sort_values()

In [9]:
# Let's check NAs once again

(df_unstacked.isna().sum(axis=1) > 0).sum()/len(df_unstacked) #We see that over 96% of rows contains at least one missing value - we cannot simply delete them

### Completing the information about weather stations

Connecting previously loaded data with data from kody_stacji.csv file

In [10]:
# Loading the data file

path_codes = os. getcwd() + '/data/kody_stacji.csv'
df_codes = pd.read_csv(path_codes, encoding = "ISO-8859-2", sep = ";", decimal = ',')

In [11]:
# Let's look and the data and change names and formats

df_codes.info()
display(df_codes)

df_codes.columns = ["id", "name", "lat", "long"]
df_codes = df_codes.astype({'id':'int', 'name': 'str', 'lat': 'str', 'long': 'str'})

In [12]:
# Merging the data about the weather and about the weather stations

df_ws_merged = pd.merge(df_unstacked, df_codes, on ='id', how ='left') 

# Checking if it was correct

# df_merged['Lat'].isnull().values.any()
# df_merged['Long'].isnull().values.any()
# df_merged['Name'].isnull().values.any()

# No nulls so the data was merged okay

In [13]:
# Let's also add data columns

df_ws_merged['year'] = df_ws_merged['date'].dt.year 
df_ws_merged['month'] = df_ws_merged['date'].dt.month
df_ws_merged['day'] = df_ws_merged['date'].dt.day

df_ws_merged

### Checking the data correctness

Before proceding to data aggregation and further analysis, I wanted to check the data correctness. To do that, I have taken a look at the basic statistics for the weather variables.

In [14]:
pd.options.display.float_format = '{:.2f}'.format
df_ws_merged.drop(['id', 'day', 'month', 'year'], axis = 'columns').describe()

Based on the table above, we can identify the following problems with the data:
    
- Variable C - average wind speed: we have very large maximum wind speeds. I assume that the unit for variable C are m/s (based on real data for Poland), so the wind speed of 1000 m/s is unreal and it is likely a incorrect observation.
- Variable E: relative air humidity: according to the definition, relative air humidity cannot be lower than 0 (source: https://en.wikipedia.org/wiki/Humidity#Relative_humidity, https://www.chicagotribune.com/news/ct-xpm-2011-12-16-ct-wea-1216-asktom-20111216-story.html). 

The rest of the variables seem reasonable.

Let's first tackle problems with variable C.

In [15]:
# Based on the statistics, I assume that the problem with high values is in few outliers.
# Let's delete all observations with C value higher than 100 m/s which is approximately the highest wind speed ever recorded in Poland.

df_ws_merged.C = np.where(df_ws_merged.C > 100, np.NaN, df_ws_merged.C)

Now let's correct variable E.

In [16]:
# I assume that E values below 0 are incorrect. I trasform them into values above 0.
# I delete values equal to 0.

df_ws_merged.E = np.where(df_ws_merged.E < 0, df_ws_merged.E*-1, df_ws_merged.E)
df_ws_merged.E = np.where(df_ws_merged.E == 0, np.NaN, df_ws_merged.E)

### Aggregating the data about weather stations

In the original file the data was provided at the level of hours. However, we have lot's of missing data and analyzing data at such a small level is not sensible. 

In this analysis I decided to aggregate the data at the month level, as the other data about BTS stations malfunctions is also aggregated at this level. Moreover, as not only the weather itself, but also its variability may, in my opinion, influence the functioning of BTS stations, I took into account both mean and standard deviation of weather variables across the month.

I have chosen mean as a general indicator of this weather parameter throught the month. I also took standard deviations as a measure of variability of that parameter.

In [17]:
# Aggregating and calculating monthly means of weather variables

df_month_mean = df_ws_merged.loc[:, df_ws_merged.columns != 'day'].groupby(['id', 'name', 'lat', 'long', 'year', 'month']).mean(numeric_only = True)
df_month_mean.reset_index(inplace = True)


# Aggregating and calculating monthly standard deviations of weather variables

df_month_std = df_ws_merged.loc[:, df_ws_merged.columns != 'Day'].groupby(['id', 'name', 'lat', 'long', 'year', 'month']).std(numeric_only = True)
df_month_std.reset_index(inplace = True)

In [18]:
# Let's merge both datasets

df_ws = pd.merge(df_month_mean, df_month_std, on = ['id', 'name', 'lat', 'long', 'year', 'month'], how = 'left', suffixes = ('_mean', '_std'))
display(df_ws)

To make the analysis more clear, I would like to focus only on one year. In the data we have two years: 2017 and 2022. 

Let's check if both years are similiar in terms of mean values of monthly weather variables using ANOVA.

In [19]:
import scipy.stats as stats

years = [2017, 2022]

for variable in df_ws.columns[6:11]:  # Get the parameter columns
    data = [df_ws[df_ws['year'] == year][variable].dropna() for year in years]
    f_statistic, p_value = stats.f_oneway(*data)
    print(variable, p_value) 
    
# Based on p-value we cannot reject the null hypothesis that the corresponding variable means for each year are the same. 
# It means that we cannot see statistically significant differences in monthly mean values of parameters between the years.

As both years seem similiar, I choose the most recent year of 2022 and the rest of my analysis will be based only on that data.

In [20]:
df_ws2022 = df_ws[df_ws['year'] == 2022]
df_ws2022.drop('year', axis = 'columns', inplace = True)
df_ws2022

### Handling missing values

Now let's handle the missing values in the dataset. It is important for later analysis, connecting the weather data with BTS stations and modelling.

In [21]:
sum (df_ws2022.groupby(['id', 'name', 'lat', 'long']).count().month < 6)
# We see that for 20 stations we do not every at least one parameter value for all months.


# For now, let's create new rows for those months and fill them with NAs

all_combinations = pd.MultiIndex.from_product([df_ws2022['id'].unique(), range(1, 13)], names = ['id', 'month'])
all_data = pd.DataFrame(index = all_combinations).reset_index()

df_ws2022 = pd.merge(all_data, df_ws2022, on = ['id', 'month'], how = 'left')

Let's check the correlations between variables - we are working here on mean values for months, as standard deviation have different interpretation.

In [22]:
corr = df_ws2022[['A_mean', 'B_mean', 'C_mean', 'D_mean', 'E_mean']].corr()
corr.style.background_gradient(cmap = 'coolwarm', axis=None)

# As one could predict, A (air temperature) and B (ground temperature) are very highly correlated. 
# Therefore, parameter B is not that important for our analysis, as long as we have value of parameter B and vice versa.

In [23]:
# Let's explore further how missing value structure looks like

import missingno as mno

(df_ws2022.isna().sum(axis=1) > 0).sum() / len(df_ws2022)
# We still have almost 60% of rows with at least one missing value of parameter

mno.matrix(df_ws2022[['A_mean', 'B_mean', 'C_mean', 'D_mean', 'E_mean']])
# We also see that most observations with missing values have every observation missing except for D parameter value.

In [24]:
# Let's check number of missing values by ID

NAs_by_ID = df_ws2022.drop(['id', 'name', 'lat', 'long'], 1).isna().groupby(df_ws2022.id, sort = False).sum(numeric_only = True).reset_index()
NAs_by_ID['unhelpful_parameters'] = (NAs_by_ID.iloc[:, 2:] == 12).sum(axis=1) / 2 # Count how many parameters are missing for all months for specific ID

Weather stations for which we have data for only one or two parameters for the whole year (i.e. for a particular parameter they have NAs in all months) are little helpful in our analysis. In theory, we could extrapolate the value of missing parameters, for example by looking at other nearby stations or trying to derive it from the values of other parameters. However, it would be complicated and time-consuming and/or without the guarantee of good results (correlations are not that high except for A and B). I also checked if maybe we can subsitute it with data from the 2017, but from my brief checking (code not included here, I just changed the selected year above and performed the same calculations) we wouldn't get much more information. 

Therefore I decided to drop every station that have less than 4 out of 5 parameters (or 3 of 4 if we assume that A and B are carring the same information).

In [25]:
# Let's check for how many observations we have less than 4 out of 5 parameters present - we are going to delete those 

sum(NAs_by_ID['unhelpful_parameters'] >= 3) # Unfortunately it is 300 stations which is more than a half of all our data :( 

# As we have seen in correlogram A can subsitute for B and vice versa. Let's check if we could have such situation. 
NAs_by_ID[(NAs_by_ID['A_mean'] + NAs_by_ID['B_mean']) == 12]
good_A_B = np.array(NAs_by_ID[(NAs_by_ID['A_mean'] + NAs_by_ID['B_mean']) == 12]['id'].drop(NAs_by_ID.index[137]))

# Deleting stations with more than 3 unhelpful parameters, except these exceptions above
bad_parameters = np.array(NAs_by_ID[NAs_by_ID['unhelpful_parameters'] >= 3]['id'])
df_ws2022d = df_ws2022[~df_ws2022['id'].isin(bad_parameters) | df_ws2022['id'].isin(good_A_B)].copy()
df_ws2022d.reset_index(inplace = True)

(df_ws2022d.isna().sum(axis=1) > 0).sum() / len(df_ws2022d)
# Now we only have around 10% of parameters missing

The missing values for A can be extrapolated from missing values of B and vice versa. Here I am using, well, maybe not the most correct approach but simple and I would say effective enough for those few observations (I could do PCA and take the first principal component as some general indicator of temperature, but it would be more complex and I would prefer for now to stick with original values). 

I check how, in most cases, the relationship between A and B looks like and infer the value of one of the missing parameters from it if possible.

In [26]:
coef_mean = (df_ws2022d['A_mean'] / df_ws2022d['B_mean']).median()
coef_std = (df_ws2022d['A_std'] / df_ws2022d['B_std']).median()

df_ws2022d['A_mean'] = df_ws2022d.apply(lambda row: row['B_mean']*coef_mean if np.isnan(row['A_mean']) else row['A_mean'], axis=1)
df_ws2022d['B_mean'] = df_ws2022d.apply(lambda row: row['A_mean']/coef_mean if np.isnan(row['B_mean']) else row['B_mean'], axis=1)

df_ws2022d['A_std'] = df_ws2022d.apply(lambda row: row['B_std']*coef_mean if np.isnan(row['A_std']) else row['A_std'], axis=1)
df_ws2022d['B_std'] = df_ws2022d.apply(lambda row: row['A_std']/coef_mean if np.isnan(row['B_std']) else row['B_std'], axis=1)

For the rest of the missing values, I fill them based with median value for a particular parameter for Poland for a particular month.

In [27]:
month_medians = df_ws2022d.groupby('month').median(numeric_only = True)

def fill_NAs(row, variable):
    if pd.isnull(row[variable]):
        return month_medians.loc[row['month'], variable]
    else:
        return row[variable]

for variable in df_ws2022d.columns[6:]:
    df_ws2022d[variable] = df_ws2022d.apply(lambda row: fill_NAs(row, variable), axis=1)

In [28]:
df_ws2022d[df_ws2022d.isna().any(axis=1)] 
# There were some missing Name, Lat and Lot values from the previous operation of filling in NAs for missing months.
# I checked them and they are okay, not the result of an error, so I am filling them in.

df_ws2022d.fillna(method='ffill', inplace = True)

In [29]:
df_ws2022d

### Loading the data on BTS

Loading the data from bts.cv file

In [30]:
path_bts = os. getcwd() + '/data/bts.csv'

df_bts = pd.read_csv(path_bts, decimal = ',', encoding = "utf-8")
df_bts.info()

In [31]:
# Changing column names and types

df_bts.columns = ["bts_id", "provider", "technology", "location"]
df_bts = df_bts.astype({'provider': 'str', 'technology': 'str', 'location': 'str'})
df_bts.standard = df_bts.technology.str.strip()
df_bts

Creating binary variables representing technologies supported by stations.

In [32]:
df_bts['technology'] = df_bts['technology'].apply(lambda x: x.split())
df_bts
unique_names = set(name for names in df_bts['technology'] for name in names)

for name in unique_names:
    df_bts[name] = df_bts['technology'].apply(lambda x: 1 if name in x else 0)

df = df_bts.drop(['technology'], axis=1)
df_bts

Loading the coordinates of cities where BTS stations are located

Unfortunately, due to large amount of locations to process, mapping all coordinates using Geopy took quite a long time and quickly I got timeout. This is why my primarly method of mapping coordinates of BTS stations was using an exteranl source - the website https://astronomia.zagan.pl/art/wspolrzedne.html. I copy pasted the data to Excel and tranformed it there as it was much quicker and simpler than direct webscrapping. Next, I loaded this Excel file miejscowosci_lista.csv  here and did the coordinates mapping. Only when there was some problem with mergining the data I used Geopy.

Some of the city names in the bts.csv file were ambiguous as in Poland we can have more than one city with the same name (there are unique only to powiats). In this case I assumed that I can assign the first coordinates by alphabetical order of powiat name that the city belongs to. 

I saved the data in miejscowosci_lista.csv. file.

In [33]:
# Loading the preprepared file based on information from the website https://astronomia.zagan.pl/art/wspolrzedne.html

path_cities = os. getcwd() + '\\miejscowosci_lista.csv'

df_cities_list = pd.read_csv(path_cities, encoding = "utf-8")
df_cities_list
df_cities_list.drop_duplicates(subset = ['location'], inplace = True) # Powiat names were deleted from location names before, here I delete duplicates created 

In [34]:
# Changing data type

df_cities_list = df_cities_list.astype({'location': 'str', 'long': 'str', 'lat': 'str'})
df_cities_list

# Ujednolicanie nazw i zapisu

df_cities_list.location = df_cities_list.location.str.strip()
df_bts.location = df_bts.location.str.strip()

df_cities_list = df_cities_list.replace('°', '.', regex = True)

Mapping

In [35]:
df_bts_coor = pd.merge(df_bts, df_cities_list, on ='location', how = 'left')
df_bts_coor 

In [36]:
# If location could not be found, I used Geopy mapping

lista_miast = df_bts_coor[df_bts_coor.long.isnull()].location.unique()

from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent = "request_Monika")

latitudes = [geolocator.geocode(miasto + ', Polska').latitude for miasto in lista_miast]
longitudes = [geolocator.geocode(miasto + ', Polska').longitude for miasto in lista_miast]

In [37]:
geopy_mapped = pd.DataFrame({'location': lista_miast, 'long2': longitudes, 'lat2': latitudes})

Merging coordinates data from the website and from the geopy mapping

In [38]:
df_bts_coor_merged = pd.merge(df_bts_coor, geopy_mapped, on ='location', how ='left') 

df_bts_coor_merged['long'].fillna(df_bts_coor_merged['long2'], inplace = True)
df_bts_coor_merged['lat'].fillna(df_bts_coor_merged['lat2'], inplace = True)
df_bts_coor_merged = df_bts_coor_merged.drop(['lat2', 'long2'], axis = 1)
df_bts_coor_merged

### Assigning BTS stations data on the weather

We have already loaded the data on weather stations and data on BTS stations. To analyze the relationship between the weather and BTS station malfuntioning, we need to assing every BTS station a weather stations from which we can infer the weather at the location of this particular BTS station. 

Of course, we could assign more than one weather station to the BTS station and take, let's say a mean of weather variables weighted by the distance of particular weather station to that weather location. However, it seems like as unnecceary complication given the time and scope of this project. Although assigning each BTS station just one weather station means that we will have multiple BTS stations with the same weather data.

Let's prepare df with weather data

In [39]:
# Ujednolicenie współrzędnych do systemu dziesiętnego
def coordinate_split(x):
    deg, minutes, seconds =  x.strip().split()
    return (float(deg) + float(minutes)/60 + float(seconds)/(60*60))

df_ws2022d[['lat', 'long']] = df_ws2022d[['lat', 'long']].applymap(coordinate_split)

stacje_pogoda = df_ws2022d[['id', 'lat', 'long']].drop_duplicates(ignore_index = True)
stacje_pogoda = stacje_pogoda.astype({'lat': 'float', 'long': 'float'})

df_bts_coor_merged = df_bts_coor_merged.astype({'long': 'float', 'lat': 'float'})
df_bts_coor_merged

Let's calculate the distance between weather stations and BTS stations using the Haversin formular. Next, each BTS station we will asssign weather station that is in the closest distance to it.

In [40]:
from math import radians, sin, cos, sqrt, atan2
from scipy.spatial.distance import cdist

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2) ** 2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# Function to find the nearest location in df2 for a given location in df1
def find_nearest_location(row):
    distances = stacje_pogoda.apply(lambda x: haversine(row['lat'], row['long'], x['lat'], x['long']), axis=1)
    min_distance_index = distances.idxmin()
    nearest_location = int(stacje_pogoda.loc[min_distance_index, 'id'])
    min_distance = distances[min_distance_index]
    return nearest_location, min_distance  

df_bts_coor_merged[['near_w_station', 'dist_near_w_station']] = df_bts_coor_merged.apply(find_nearest_location, axis=1, result_type='expand')

In [41]:
df_bts_coor_merged['dist_near_w_station'].hist()

In [42]:
df_bts_coor_merged['near_w_station'].value_counts().hist()

We see that the assigment went quite okay - most BTS stations are within 50 km of their assigned weather stations, there are few examples above that. Moreover, as we have more BTS stations than weather stations, some weather stations were assigned to more than 250 weather stations. However, it would be hard to avoid such situation, althought it would be interesting to check later, how it corresponds with anomalies.

In [43]:
# Merging the data - each BTS station we assigned the nearest location of weather station. 
# Now, we transfer the corresponding data on the weather to each BTS station.

df_bts_weather = pd.merge(df_bts_coor_merged, df_ws2022d, left_on = 'near_w_station', right_on = 'id', how = 'outer')
df_bts_weather.dropna(subset = ['bts_id'], inplace = True)
df_bts_weather

In [44]:
pd.set_option('display.max_columns', None)
df_bts_weather

### Adding the data on BTS malfunctioning

Loading the data from bts_anomalie_pct.xlsx file

In [45]:
path_bts_anomalies = os.getcwd() + '/data/bts_anomalie_pct.xlsx'

df_bts_anomalies_raw = pd.read_excel(path_bts_anomalies)
df_bts_anomalies_raw

In [46]:
# Let's focus on the year 2022
df_bts_anomalies_raw = df_bts_anomalies_raw.iloc[:, 13:]

Stacking for easier merging 

In [47]:
anomalies_stacked = df_bts_anomalies_raw.stack()

In [48]:
df_bts_anomalies = pd.DataFrame(anomalies_stacked)
df_bts_anomalies.reset_index(inplace = True)
df_bts_anomalies.columns = ['bts_id', 'month', 'anomalies']
df_bts_anomalies['month'] = df_bts_anomalies['month'].str.split('-').str[1].astype(int)
df_bts_anomalies

In [49]:
# Checkign NAs
df_bts_anomalies.isna().sum()

Merging

In [50]:
df_main = pd.merge(df_bts_weather, df_bts_anomalies, how = 'left', on = ['bts_id', 'month'])

In [51]:
# Let's delete unnessesary columns and arrange it better

df_main.drop(['technology', 'day', 'index', 'near_w_station'], axis = 'columns', inplace = True)
df_main.rename(columns = {'long_x': 'long_bts', 'lat_x': 'lat_bts', 'lat_y': 'lat_w_station', 'long_y': 'long_w_station', 'id': 'w_station_id', 'location': 'location_bts', 'name': 'location_w_station'}, inplace = True)
df_main = df_main.astype({'bts_id': 'int', '5G': 'bool', 'UMTS': 'bool', 'IOT': 'bool', 'LTE': 'bool', 'GSM': 'bool'})

In [52]:
# pd.set_option('display.max_columns', None)
df_main

In [53]:
# pd.reset_option('display.max_columns')

We obtained our final dataset with all information intresting for us. Let's save it for further use.

In [54]:
df_main.to_pickle('df_data_final.pkl')
df_main = pd.read_pickle('df_data_final.pkl')

## Data analysis

At the previous stage, I already adressed the following problems:
- data correctness - I deleted or corrected likely errors in the weather data
- handling missing data - I deleted weather observations where number of missing values was too high; for the rest of observations in filled it in with median parameter values for a particular month. At this point we have no missing data
- handling duplicates - they were deleted
- choosing data scope - our data is aggregated at the month level in term of mean values and standard deviations. I interpret mean monthly values of parameters, for example A_mean as general indicator of the variable for the paricular station and month. I interpret montly standard deviations of parameters, for example A_std as indicator of variability of the variable for the paricular station and month. 

Now we can proceed to the data analysis.

### Preliminary analysis for the weather stations data

Taking into account our main goal i.e. analyzing the relationship between the weather and BTS station malfuntioning, we are interested only in the weather data from the weather stations that were "assigned" to one of the BTS stations. We could theoretically analyze all the stations across Poland and the results would be probably similiar, but for clarity let's focus only on weather stations assigned to BTS stations.

At some points during preprocessing we already calculated basic statistics and correlations, but let's do it again for the weather data in the final dataset. 

(*I am aware that after aggregation, we obtained montly means and stds based on separate hour values i.e. our raw data. Now, we will be calulaing for example, mean of those means which probably won't be exactly equal to the mean which could be calculated based on observations for separate hours, the same for standard deviations etc.). However, I treat it as good enough approximation for later analysis and those variables still have explanotary power*).

Exctracting the weather data from the main file, so that we do not count duplicate stations as separate observations.

In [55]:
df_weather = df_main.drop_duplicates(subset = ['w_station_id', 'month']).reset_index(drop = True).iloc[:, 11:]
df_weather

**Basic statistics** of weather variables and BTS malfunctioning (anomalies) across all months and all BTS stations

In [1]:
df_weather.iloc[:, 4:].describe()

NameError: name 'df_weather' is not defined

Looking at the basic statistics, we can also see high correlation or likeliness between A and B parameters. Values of all parameters seem quite likely in terms of correspondence with real life.

Now let's look at the **Pearson correlations** for the weather variables (numerical).

In [57]:
corr = df_weather.iloc[:, 4:].corr()
corr.style.background_gradient(cmap = 'coolwarm', axis=None)

As we are anlyzing the weather data, we see some obvious relationships between variables. Most of them could be probably explained by science, if one is a weather expert. For example, wind conditions and humidity are generally dependend on the temperature and pressure of the system. At the first glance, we see very high positive linear correlations between A (air temperature) and B (ground temperature) variables, both between their mean values and between their standard deviations (which is of course expected). For stds, we have high negative correlation for B_std and E_std, which is interesting. The rest of stds are not that significantly correlated. For the rest, correlations are not that strong.

Moreover, there are quite significant correlations between a particular variable mean and its standard deviation. Although under normal distribution, mean and standard deviation should be independent, here we observe that for variables A, B, C and D the higher the value of the parameter in a particular month, the higher its variability. For variable E it is the opposite.

Unfortunately, all analyzed variables are not at all linearly correlated with percent of anomalies, which is our target variable. It suggest that doing simple linear regression on weather variables to predict the anomalies might not be sufficient. However, we still not taken into account categorical variables, such as months or stations itself. 


Now let's look at **mean air temperature and its variability** across the months at the level of Poland (all BTS stations).

In [58]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker    
import calendar

month_labels = [calendar.month_abbr[x] for x in range(1, 13)]

pom = df_weather.groupby('month').mean(numeric_only = True)

fig, ax = plt.subplots(figsize=(8, 4))
plt.plot(month_labels, pom['A_mean'], label = "Mean monthly temperature", color = 'blue') 
plt.plot(month_labels, pom['A_std'], label = "Mean standard deviation of temperature throught the month", linestyle = 'dotted')
plt.gca().yaxis.set_major_formatter(mticker.FormatStrFormatter('%d°C'))
plt.title("Temperature in BTS stations in 2022")
plt.legend(loc = 'lower center', bbox_to_anchor=(0.5, -0.3)) 
plt.show()

According to expectations, we see the highest montly mean temperatures of around 18-20°C in summer months: August, July and June. On the other hand, the lowest temperatures of 0°C we have in winter months: January and December. Mean standard deviation of temperatures are pretty constant throught the year and are of around 3-5°C.

Now let's look at mean **wind speed and total daily percipitation** across the months at the Poland level (all BTS stations). 

In [59]:
fig, ax = plt.subplots(figsize=(8, 4))
plt.plot(month_labels, pom['C_mean'], label = "Mean monthly wind speed", color = 'black') 
plt.bar(month_labels, pom['D_mean'], label = "Mean daily rainfall during the month")
plt.ylabel("Wind in km/h or rainfall in mm") # Założyłam jednostki
plt.title("Weather variables measured by weather stations in 2022")
plt.legend(loc = 'lower center', bbox_to_anchor=(0.5, -0.3)) 
plt.show()

We see the highest daily percipitation in July and September and the lowest in March. The wind is the strongest in winter, in January and February.

### Preliminary analysis for the BTS stations data

Let's focus now on the BTS stations data.

In [60]:
df_main.provider.value_counts().plot(kind = 'barh', title = 'Number of BTS stations by provider')

In [61]:
import matplotlib.ticker as mtick

fig, ax = plt.subplots(figsize=(8, 4))

technologies = ['LTE', 'GSM', 'UMTS', '5G', 'IOT']
percentages = [(df_main[technology].value_counts(normalize = True) * 100)[1] for technology in technologies]

plt.barh(technologies, percentages)
plt.gca().xaxis.set_major_formatter(mtick.PercentFormatter())
plt.title("Percentage of all BTS stations using particular technology")

Also, let's look at the distribution of BTS failure rates.

In [62]:
df_main.anomalies.hist(bins = 16)
plt.xlabel('Percentage of anomalies')
plt.ylabel('Nuber of observations')
plt.title('Distribution of percentage of anomalies across BTS stations')

Well, as it was said, the data was generated randomly which could probably explain the uniform distribution of anomalies. All the possible values range from 0.00 to 0.15 by 0.01. In real life, one would probably expect something more normal looking, for example that a high rates of anomalies would be less frequent. 

We can also check if generally anomalies could be dependent on the month using one-way ANOVA, but we see that they are not.

In [63]:
anomalies_by_month = df_main[['month', 'anomalies']]

anova_abm = stats.f_oneway(*[group['anomalies'] for name, group in anomalies_by_month.groupby('month')])
print(anova_abm.pvalue)

# P-value is much higher than 0.05 so we cannot reject the null hypothesis that the mean values of anomalies are statistically equal across the months.

### Comparing Orange and other providers

Let's compare anomalies for Orange stations and its competitors.

In [64]:
providers = df_main.drop(['bts_id', 'long_bts', 'lat_bts', 'month', 'w_station_id'], axis = "columns").groupby('provider').mean()
providers['bts_id_count'] = df_main.provider.value_counts()
providers

# The table is pretty straightfoward, so I feel no need to visualize it.

Looking at the table, we can see that all providers have, in fact, very similiars networks in distributions of technologies they use and weather conditions at the stations.

### Identification of stations that are exposed to the most extreme weather changes

As I aggregated data on the monthly levels, we will focus on those statistics; however, I acklowledge that it could be also beneficial to have the data at lower level of aggregation, for example to check the number of separate extreme weather events throught the month that usually last few hours. Still, as standard deviation was calculated based on those raw single observation on the level of hours, we have some information preserved. I am focusing here on standard deviation as an indicator of weather changes/its variability.

Here I am first identifing **weather stations** among those assigned to BTS, so using the df_weather dataframe. As was showned before, each weather station can have up to 250 BTS stations assigned to it.

In [65]:
# The stations with largest A_std are actually the same as the largest variations in B_std

top3_AB_std = df_weather.nlargest(3, 'A_std')
top3_AB_std

In [66]:
top3_C_std = df_weather.nlargest(3, 'C_std')
top3_C_std
# The largest variation is for Śnieżka which could be explained by its special location in the mountains.

In [67]:
top3_D_std = df_weather.nlargest(3, 'D_std')
top3_D_std

Now let's get all BTS stations that are assigned to one of the above weather stations identified as those with the highest variability of weather variables. We see that we have quite a lot of those stations.

In [68]:
weather_changes_stations = list(set(top3_AB_std['w_station_id'].to_list() + top3_C_std['w_station_id'].to_list() + top3_D_std['w_station_id'].to_list() ))

filtered_data = df_main[df_main['w_station_id'].isin(weather_changes_stations)]
bts_ids_weather_changes = list(set(filtered_data['bts_id']))
print(*bts_ids_weather_changes)

### Visualization of stations on the map

Loading the map of Poland with voivodeships borders. I downloaded the shapefile from https://gis-support.pl/baza-wiedzy-2/dane-do-pobrania/granice-administracyjne/

In [69]:
import geopandas as gpd 

path = os.getcwd() + '\wojewodztwa\wojewodztwa.shp'

polska = gpd.read_file(path).to_crs(epsg = 4326)
polska.crs

Connecting the points from our dataframe to points on the map

In [70]:
df_main_bts = df_main.copy()
df_main_ws = df_main.copy()

In [71]:
from shapely.geometry import Point

geometry_bts = [Point(xy) for xy in zip(df_main_bts['long_bts'], df_main_bts['lat_bts'])]
geo_df_bts = gpd.GeoDataFrame(df_main_bts, geometry = geometry_bts, crs = 'epsg:4326')

geometry_w_station = [Point(xy) for xy in zip(df_main_ws['long_w_station'], df_main_ws['lat_w_station'])]
geo_df_w_station = gpd.GeoDataFrame(df_main_ws, geometry = geometry_w_station, crs= 'epsg:4326')

Now, let's visualize everything on the maps

In [72]:
fig, ax = plt.subplots(figsize = (7, 7))

polska.plot(ax = ax, color = 'lightgrey', edgecolor='white', linewidth=1)

geo_df_bts.plot(ax = ax, color = 'deepskyblue', marker = 'o', markersize = 7, label = 'BTS')
geo_df_w_station.plot(ax = ax, color = 'tomato', marker = '^', markersize = 9, label = 'Weather Station')

ax.set_title('BTS and Weather Station Locations')
ax.legend()

plt.show()

We see that the BTS stations are actually grouped in quite visible groups. Also we see some locations even outside Poland. Let's remind that this are locations whose coordinates we had to map ourself using city names. This might be due to artificial generation of data, actual BTS stations characteristics or - I hope not because I have checked it - possible problems with mapping :)

On the other hand, weather stations seems to be placed at random. However, let's remind that during the initial phrase we dropped almost half of weather stations due to insufficient data, and then some of the remaining weather station might have not been used as they were not close to any BTS station. So we do not know the initial distribution of those stations across the map.

Let's also try to visualize some weather parameters. For example, let's visualize which weather station experience the highest temperatures during August (the warmest month).

In [73]:
import matplotlib.colors as mcolors

fig, ax = plt.subplots(figsize=(7, 7))
polska.plot(ax = ax, color='lightgrey', edgecolor = 'white', linewidth = 1)

df_month_8 = df_main[df_main['month'] == 8]

max_a_mean_month_8 = df_month_8.groupby('w_station_id')['A_mean'].max()
geo_df_w_station = geo_df_w_station.merge(max_a_mean_month_8, left_on='w_station_id', right_index=True, suffixes=('', '_max'))

# Plot Weather Stations with colors based on A_mean_max
cmap = plt.cm.get_cmap('Reds')
normalize = mcolors.Normalize(vmin=geo_df_w_station['A_mean_max'].min(), vmax=geo_df_w_station['A_mean_max'].max())
colors = [cmap(normalize(value)) for value in geo_df_w_station['A_mean_max']]

geo_df_w_station.plot(ax = ax, color = colors, marker = 'o', markersize = 14)

# Creating custom color map legend
sm = plt.cm.ScalarMappable(cmap = cmap, norm = normalize)
sm.set_array([]) 
cbar = plt.colorbar(sm, ax=ax, orientation = 'vertical', fraction = 0.03, pad = 0.1)
cbar.set_label('Mean temperature')

ax.set_title('Mean temperature for weather stations in August')
plt.show()

### Assesing the impact of weather on BTS malfunctioning

In [74]:
# Splitting the dataset into dependent and independ variables

X = df_main[['bts_id', 'month', 'UMTS', 'IOT', 'LTE', '5G', 'GSM', 'A_mean', 'C_mean', 'D_mean', 'E_mean', 'A_std', 'C_std', 'D_std', 'E_std']].set_index(['bts_id', 'month']) #Not including B
X = pd.get_dummies(data = X, columns = ['UMTS', 'IOT', 'LTE', '5G', 'GSM'], drop_first = True)
Y = df_main['anomalies']

In this project we are dealing with the panel data which means that we have multiple units which we observe at multiple points in time. From the statistical perpective, panel data could be harder to deal with due to autocorrelation of errors from the time dimension. Therefore, it is better to use regressions models specifically dedicated to panel data.

There are three main panel data models: pooled OLS, fixed effects and random effects.

Let's start with **pooled OLS** which is the most basic one and corresponds to standard OLS, just done on the panel data.

In [75]:
df_panel = df_main[['bts_id', 'month', 'UMTS', 'IOT', 'LTE', '5G', 'GSM', 'A_mean', 'C_mean', 'D_mean', 'E_mean', 'A_std', 'C_std', 'D_std', 'E_std', 'anomalies']].set_index(['bts_id', 'month'])
df_panel = pd.get_dummies(data = df_panel, columns = ['UMTS', 'IOT', 'LTE', '5G', 'GSM'], drop_first = True)

months = df_panel.index.get_level_values('month').to_list()
df_panel['months'] = pd.Categorical(months)

In [76]:
from linearmodels import PooledOLS
import statsmodels.api as sm

X = df_panel.loc[:, df_panel.columns != "anomalies"]
X = sm.add_constant(X)
Y = df_panel['anomalies']

mod = PooledOLS(Y, X)
pooledOLS_res = mod.fit(cov_type = 'clustered', cluster_entity=True)

# Store values for checking homoskedasticity graphically
fittedvals_pooled_OLS = pooledOLS_res.predict().fitted_values
residuals_pooled_OLS = pooledOLS_res.resids

Now let's check whether this model is correct. For Pooled OLS to be correct we have to have:
- homoskedasticity of errors (constant variance) 
- no autocorrelations of erros (no correlations between errors in different terms)

Let's check homoskedasticity first using visualization and statistical tests

In [77]:
# Plotting errors for visual inspection

import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.scatter(fittedvals_pooled_OLS, residuals_pooled_OLS, color = 'blue')
ax.axhline(0, color = 'r', ls = '--')
ax.set_xlabel('Predicted Values', fontsize = 15)
ax.set_ylabel('Residuals', fontsize = 15)
ax.set_title('Homoskedasticity Test', fontsize = 30)
plt.show()

# Errors seems to be equally distributed (as equal as generated data can be)

In [78]:
# Now let's check homoskedasticity using the formal tests

from statsmodels.stats.diagnostic import het_white, het_breuschpagan

# White-Test
white_test_results = het_white(pooledOLS_res.resids, X)
labels = ['LM-Stat', 'LM p-val', 'F-Stat', 'F p-val'] 
print(dict(zip(labels, white_test_results)))

# Breusch-Pagan-Test
breusch_pagan_test_results = het_breuschpagan(pooledOLS_res.resids, X)
print(dict(zip(labels, breusch_pagan_test_results)))

# In both tests p-value is higher than 0.05 -> we cannot reject the null hypothesis that errors are homoskedastic

Let's also check the autocorrelation using Durbin-Watson test

In [79]:
from statsmodels.stats.stattools import durbin_watson

# Durbin-Watson test
durbin_watson_test_results = durbin_watson(pooledOLS_res.resids) 
print(durbin_watson_test_results)

# The value of Durbin-Watson test is very close to 2 which suggests no autocorrelation

Suprisingly, we found that the Pooled OLS model fulfill approriate assumptions. Suprisingly because it is very hard to fullfill these assumptions on real-world data, but here it can be explained since the data was generated. Therefore, our Pooled OLS is correct and could derive statistical inference. It could be beneficial to compare Pooled OLS also with other models (fixed and random effects) but due to lack of time, let's stick with that.

Let's check the summary.

In [80]:
print(pooledOLS_res)

It seems that all the variables are actually insignificant! They all have p-values greater than 0.05, and much greater in most cases. This also includes all weather variables. We have seen from the correlation table that, in fact, there are little linear relationship between them and the anomalies. However, I still hoped than when controlling for other variables, some relationship could become prominent. 

In this case we might try to also add square roots of weather variables - maybe anomalies are more prominent in extreme weather?

In [81]:
# Adding square roots of variables

# I tried to did that code but got error "exog does not have full column rank" which suggest that the created columns were linear combiantions of other coluns

# columns_to_square = ['A_mean', 'C_mean', 'D_mean', 'E_mean']
# X2 = sm.add_constant(X.assign(**{f'{col}^2': lambda x: x[col] ** 2 for col in columns_to_square}))

X['A_mean2'] = X['A_mean']**2
# I also tried other variables separately, but only A was stastically significant

In [82]:
# Building a new model

mod2 = PooledOLS(Y, X)
pooledOLS_res2 = mod2.fit(cov_type = 'clustered', cluster_entity=True)

# I do present here checking the assumptions once again as the data has not changed that much. All assumptions still hold.

In [83]:
print(pooledOLS_res2)

We see that only the air temperature (variable A) and its square root is significant with p-values below 0.05 (as mentioned in the comments, I also tested adding square roots of other variables but they were insignificant). However, its coefficient is very small which suggests that still, temperature does not affect anomalies in any significant way. Also, the April (months.4) variable became significant, but at the margin (0.0489) and in previous regression it was pretty insignificant, so I would say this is within statistical error. Moreover, the R^2 of the model is very small as well. 

To sum up, based on the pooled OLS models I assess that **the weather (and other variables, such as months or technology) have no relationship with anomalies in BTS stations** (I am using pooled OLS so from the stastical perpective I cannot say anything about the casual relationship between those variables).

## Training machine learning models

### Clustering - identification of weather stations with similiar conditions

As I mentioned before, each weather station has assigned multiple BTS stations. Therefore, clustering BTS stations based on weather conditions would be quite poitless, as obviously all BTS stations which were assigned the same weather stations would be clustered together. 

This is why I decided to cluster only the weather stations. If it would be neccesary, one can easily extract BTS stations assigned to weather stations in each cluster. Although we are losing the data on anomalies, well - as we have seen before there is little relationship between the weather and anomalies.

#### Preparing the data

In [84]:
# We are using the dataframe df_weather created earlier. Let's unstack it and create new variables for each month.

df_weather_clus_pom = df_weather.drop(['location_w_station', 'lat_w_station', 'long_w_station', 'anomalies'], axis = 1)

df_weather_clus = df_weather_clus_pom.pivot(index = ['w_station_id'], columns = ['month'])
df_weather_clus.columns = ['_'.join(map(str, col)).strip() for col in df_weather_clus.columns.values]
df_weather_clus.reset_index(inplace=True)

df_weather_clus

In [85]:
# Let's select the features

X = df_weather_clus.iloc[:, 1:]

In [86]:
# Let's normalize them

from sklearn import preprocessing

std_scale = preprocessing.StandardScaler().fit(X)
X = std_scale.transform(X)
normalized_X = pd.DataFrame(X, columns = df_weather_clus.iloc[:, 1:].columns)

By unstacking monthly data we have obtained looots of features. I tried to reduce their number, but correlations between them were not obvious. Additionaly, when I tried the PCA, I got that 

We need to reduce the number of features. 

In [87]:
# Correlation matrix is too large to nicely display so I skipped it

# corr_clus = normalized_X.corr()
# corr_clus.style.background_gradient(cmap = 'coolwarm', axis=None)

# Briefly looking at this large matrix we see that unfortunately not all variables are correlated with each other across one
# variable type (so for example E_std_10 and E_std_8 have quite low correlation)

---

Here I tried to perform PCA. However, unfortunately I did not have time to properly interpret the principal components and use them in the further analysis, so I am just presenting what I have. 

In [88]:
from sklearn.decomposition import PCA

exp_var = []

for k in range(2, 20):

    pca = PCA(n_components = k)
    pca_features = pca.fit_transform(normalized_X)
    
    exp_var.append(pca.explained_variance_ratio_.cumsum()[-1:])

In [89]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 20).fit(normalized_X)

plt.rcParams["figure.figsize"] = (8,5)

fig, ax = plt.subplots()
xi = np.arange(1, 21, step = 1)
y = np.cumsum(pca.explained_variance_ratio_)

plt.ylim(0.0, 1.1)
plt.plot(xi, y, marker='o', linestyle='-', color='black')

plt.xlabel('Number of Components')
plt.xticks(np.arange(1, 21, step=1)) 
plt.ylabel('Cumulative variance (%)')
plt.title('The number of components needed to explain variance')

plt.axhline(y=0.95, color='grey', linestyle='--')
plt.text(1.1, 1, '95% cut-off threshold', color = 'black', fontsize=16)

ax.grid(axis='x')
plt.tight_layout()
plt.show()

We see that to properly explain the variance in the dataset, we actually need a high number of principal components which might suggest that the PCA would not be that much helpful. However, further looking into e.g. which variables contistute each principal component is needed.

---

As I unfortunately did not have time to implement PCA in  my solution, I did clustering on all the features. However, I am aware that in this situation my clustering might not be so good due to the noise.

In [90]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

kmeans_kwargs = {"init": "random", "n_init": 10, "max_iter": 300, "random_state": 42}

sse = []
silhouette_coefficients = []

for k in range(2, 15):
    
    kmeans = KMeans(n_clusters = k, **kmeans_kwargs)
    kmeans.fit(normalized_X)
    
    sse.append(kmeans.inertia_)
    
    score = silhouette_score(normalized_X, kmeans.labels_)
    silhouette_coefficients.append(score)

Let's see how many clusters we should choose.

In [91]:
# Plotting bthe SSE to check the optimal number of clusters

plt.plot(range(2, 15), sse)
plt.xticks(range(2, 15))
plt.xlabel("Number of Clusters")
plt.ylabel("Sum of Square Error (SSE)")
plt.show()

In [92]:
# Chceking also the optimal number of clusters using the silhouette score

plt.plot(range(2, 15), silhouette_coefficients)
plt.xticks(range(2, 15))
plt.xlabel("Number of Clusters")
plt.ylabel("Silhouette Coefficient")
plt.show()

Based on the SSE graph, there is no that much clear elbow visible. However, I would say that 7 clusters is a nice point where the SSE stops decreasing so rapidly.

On the other hand, looking at the silhouette coeddicient graph, we see that our assigment into groups is not good. Low coefficient values, close to 0 suggest that the clusters are not that different and might be overlapping. We should be looking at the higherst possible silhouette scores. For previously chosen 7 clusters the value is very low but still better than for most of other cluster numbers. 4 clusters, on the other hand which have the highest silhouette score after 2, are not so great based on elbow method.

This is why I would stick with 7 clusters for K-Means. 

In [93]:
kmeans = KMeans(n_clusters = 7, **kmeans_kwargs)
kmeans.fit(normalized_X)
df_weather_clus['k_means_7'] = kmeans.labels_

Let's visualize clusters on the map.

In [94]:
df_weather_clus_plot = pd.merge(df_main_ws, df_weather_clus, how = 'left', on ='w_station_id' )
df_weather_clus_plot.drop_duplicates(subset = ['w_station_id'], keep = 'last', inplace = True)

In [95]:
fig, ax = plt.subplots()

polska.plot(ax=ax, color='lightgrey', edgecolor='white', linewidth=1)

# Plotting weather stations with cluster labels
for cluster_label, color in zip(df_weather_clus_plot['k_means_7'].unique(), ['blue', 'green', 'red', 'purple', 'orange', 'brown', 'pink']):
    cluster_stations = df_weather_clus_plot[df_weather_clus_plot['k_means_7'] == cluster_label]
    ax.scatter(cluster_stations['long_w_station'], cluster_stations['lat_w_station'], label=f'Cluster {cluster_label}', color=color, marker='o', s=50)

ax.set_title('Weather Stations with K-Means Clusters')
ax.legend()
plt.legend(loc = 'center right', bbox_to_anchor=(1.5, 0.5))

plt.show()

We see that, in fact, based on geography created clusters may correspond with actual weather conditions in the stations.

### Classification model predicting a high percentage of anomaly occurences

In this part I will try to predict if a particular BTS station will experience a high percentage anomalies. I will do it based on the weather and station data. 

In the Data Analysis section we already took a look at the distribution of anomalies. As a cut-of point for classifying a percentage of anomalies as high I have chosen 0.14 which is the 94th quantile. This means that I consider the 6% of the highest percentages of anomalies as a high anomaly occurence.

#### Preparing the data

In [96]:
# from scipy import stats

# stats.percentileofscore(df_main['anomalies'], 0.14)
# df_main['anomalies'].quantile(0.9)

In [97]:
df_main['high_anomaly'] = np.where(df_main['anomalies'] > 0.14, 1, 0)
df_main['high_anomaly'].value_counts(normalize = True)

I already checked the correlation between variables in the previous steps. However, it was not very helpful. I could do a PCA for better feature selection, but due to the lack of time, I just stick with the original dataset and I only did not include B_mean and B_std due to its high correlation with A_mean and A_std.

In [98]:
X = df_main[['A_mean', 'C_mean', 'D_mean', 'E_mean', 'A_std', 'C_std', 'D_std', 'E_std', 'UMTS', 'IOT', 'LTE', '5G', 'GSM', 'month']] # Not including B
X = pd.get_dummies(data = X, columns = ['UMTS', 'IOT', 'LTE', '5G', 'GSM', 'month'], drop_first = True)
Y = df_main['high_anomaly']

In [99]:
# Splitting the data into training and testing samples so that proportion of positive and negatives are rouhly the same in each sample.

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, stratify = Y, random_state = 12345)

In [100]:
# Normalizing the data

from sklearn import preprocessing

num_cols = X_train.iloc[:, :8].columns
    
std_scale = preprocessing.StandardScaler().fit(X_train[num_cols])

X_train[num_cols] = std_scale.transform(X_train[num_cols])
X_test[num_cols]  = std_scale.transform(X_test[num_cols])

We have very low percentage of positives so our models might have problems dealing with them. This is why I rebalance the training data using SMOTE.

In [101]:
# Rebalancing training data using SMOTE

from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state = 12345)
X_train, Y_train = sm.fit_resample(X_train, Y_train)

As the main evaluation metric for the models I have chosen the F2-score. F-score is a measure of model's accuracy calculated based on its precision and recall. For F2-score, recall is considered twice as important as precision. F-scores are also suitable for imbalanced data, such as in our dataset, i.e. where number of observations in one of the classes in much higher than in the other.

I have chosen F2-score, because I believe that in our problem we are more concerned with recall than with precision. It would be better if our model misclassified some real negatives (real low anomalies) as positives (high anomalies) and we just did some additional checks of those stations, than if it misclassified real positives (real high anomalies) as negatives (low anomalies) and we were not prepared for that.

I decided to solerly focus on the F2 score metric for easier intepretability of models' results. However, given more time and resources, one should consider also other model evaluation metrics, for exampele balanced accuracy and the ROC AUC curve. 

In [102]:
# Creating a performance metric for our model

from sklearn.metrics import make_scorer, fbeta_score

f2_scorer = make_scorer(fbeta_score, beta = 2)

I am using cross-validation to test the performance of the models first on the training data and to choose appropriate parameters for the models.

In [103]:
# Importing libraries neccesary for cross validation and choosing optimal parameter values

from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

#### Logit

Let's start with simple logit model as a benchmark.

In [ ]:
from sklearn.linear_model import LogisticRegression

cv_5 = RepeatedKFold(n_splits = 5, random_state = 12345, n_repeats = 3)
logistic = LogisticRegression()
logistic_scores = cross_validate(logistic, X_train, Y_train, scoring = f2_scorer, cv = cv_5)

logistic_fitted = logistic.fit(X_train, Y_train)
print("F2 Score:", logistic_scores['test_score'].mean())

We see that the F2 score of our logit model isn't good. 

We can also try some feature engineering but since we are working on the train sample, we probably won't see an improvement of performance.

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import make_pipeline

logistic_fe = make_pipeline(SelectKBest(f_classif, k = 10), LogisticRegression())
logistic_scores_fe = cross_validate(logistic_fe, X_train, Y_train, scoring = f2_scorer, cv = cv_5)

logistic_fe_fitted = logistic_fe.fit(X_train, Y_train)
print("F2 Score:", logistic_scores_fe['test_score'].mean())

Indeed, our F2 got even a little bit worse. Let's proceed to more sophisticated models.

#### Ridge classifier

Next, let's try ridge classifier. Ridge classifier also builds a regression model, but uses different loss function than logit. It includes penalty for too high parameter values. 

In [ ]:
from sklearn.linear_model import RidgeClassifier

model_ridge = RidgeClassifier(random_state = 12345)
alpha = [0.01, 0.1, 0.25, 0.5, 0.75, 1.0, 5.0, 10.0]

grid_ridge = dict(alpha = alpha)
cv_5_3 = RepeatedKFold(n_splits = 5, n_repeats = 3, random_state = 12345)

grid_search_ridge = GridSearchCV(estimator = model_ridge, param_grid = grid_ridge, n_jobs = -1, cv = cv_5_3, scoring = f2_scorer)
grid_result_ridge = grid_search_ridge.fit(X_train, Y_train)
print("Best: %f using %s" % (grid_search_ridge.best_score_, grid_result_ridge.best_params_))

In [ ]:
print("Best: %f using %s" % (grid_search_ridge.best_score_, grid_search_ridge.best_score_))

means = grid_result_ridge.cv_results_['mean_test_score']
stds = grid_result_ridge.cv_results_['std_test_score']
params = grid_result_ridge.cv_results_['params']

for mean, param in zip(means, params):
    print("%f with: %r" % (mean, param))

We got better F2 score than for logit, but the improvement was only minor. However, due to the nature of ridge classifier, we might observe better scores on the testing data as this model is less prone to overfitting.

#### XGBoost

Now let's try XGBoost. 

As the model took me araound an hour to be calculated, I saved it in a separate file and load its results.

In [ ]:
# from xgboost import XGBClassifier

# cv_5 = KFold(n_splits = 5, random_state = 12345, shuffle = True)

# model_xgb = XGBClassifier()

# n_estimators = [100, 500]
# learning_rate = [0.001, 0.01, 0.1]
# subsample = [0.8, 1.0]
# max_depth = [3, 5, 7]
# grid = dict(learning_rate = learning_rate, n_estimators = n_estimators, subsample = subsample, max_depth = max_depth)

# grid_search = GridSearchCV(estimator=model_xgb, param_grid=grid, n_jobs=-1, cv=cv_5, scoring = f2_scorer , verbose = 1)
# grid_result = grid_search.fit(X_train, Y_train)

In [ ]:
import joblib

# Let's save the file
# joblib.dump(grid_result, 'xgb_model.sav')

# Load the model results
grid_result_xgb = joblib.load('xgb_model.sav')

In [ ]:
print("Best: %f using %s" % (grid_result_xgb.best_score_, grid_result_xgb.best_params_))

means = grid_result_xgb.cv_results_['mean_test_score']
stds = grid_result_xgb.cv_results_['std_test_score']
params = grid_result_xgb.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

We managed to achieve a significant improvement in our F2 score! The best model had it around 0.855. We could maybe achieve even better score with more complex parameter tuning, but unfortunately it is very time consuming. We might also have a problem of overfitting even with cross-validation.

#### K nearest neighbours

KNN classifier also took quite long to calculate and I am loading it from a separate file.

In [ ]:
# from sklearn.neighbors import KNeighborsClassifier

# model_KNN = KNeighborsClassifier()
# n_neighbors = range(1, 15, 2)

# grid_KNN = dict(n_neighbors = n_neighbors)

# cv_5_3 = RepeatedKFold(n_splits = 5, random_state = 12345, n_repeats = 3)

# grid_search_knn = GridSearchCV(estimator = model_KNN, param_grid = grid_KNN, n_jobs = -1, cv = cv_5_3, scoring = f2_scorer)
# grid_result_knn = grid_search_knn.fit(X_train, Y_train)

In [ ]:
# Let's save the file
# joblib.dump(grid_result_knn, 'knn_model.sav')

# Loading the file
grid_result_knn = joblib.load('knn_model.sav')

In [ ]:
print("Best: %f using %s" % (grid_result_knn.best_score_, grid_result_knn.best_params_))

means = grid_result_knn.cv_results_['mean_test_score']
stds = grid_result_knn.cv_results_['std_test_score']
params = grid_result_knn.cv_results_['params']

for mean, param in zip(means, params):
    print("%f with: %r" % (mean, param))

For KNN, we also managed to significantly improve our F2 score compared to the baseline logit model. We got the best score of 0.76. However, it still worse than our score for XGBoost.

#### Comparison of the models on the testing sample

In [ ]:
from sklearn.metrics import fbeta_score, accuracy_score, roc_auc_score

def f2_scorer_final(y_true, y_pred):
    return fbeta_score(y_true, y_pred, beta = 2)

models = [[logistic_fitted, 'Logit'],
          [logistic_fe_fitted, 'Logit Feature Selection'],
          [grid_result_ridge, 'Ridge'],
          [grid_result_knn, 'KNN'],
          [grid_result_xgb, 'XGBoost']]

for model, model_name in models:
    predicted_train = model.predict(X_train)
    predicted_test = model.predict(X_test)
    print(model_name, 
          '\n',
          'F2 score train:', "{:.4f}".format(f2_scorer_final(Y_train, predicted_train)),
          'F2 score test:', "{:.4f}".format(f2_scorer_final(Y_test, predicted_test)),
          '\n',
          'ROC AUC train:', "{:.4f}".format(roc_auc_score(Y_train, predicted_train)),
          'ROC AUC test:', "{:.4f}".format(roc_auc_score(Y_test, predicted_test)),
          '\n',
          'Accuracy train:', "{:.4f}".format(accuracy_score(Y_train, predicted_train)),
          'Accuracy test:', "{:.4f}".format(accuracy_score(Y_test, predicted_test)),
          '\n\n',
          )

We see that all of the considered classification models overfitted by a huuuge amount. To investigate it further, I added also other metrics, such as ROC AUC and Accuracy. We see that actually they are very bad, no better than a random assigment of classes. Even ridge classifier, which, in theory, is effective for overfitting, performed very bad :(

I tried to investigate this problem further, checking corectness etc. but unfortunately I have not yet managed to identify the cause of this behaviour other than maybe particular data characteristics. Generally, to avoid overfitting one might use regularization techniques (such as the afromentioned ridge), early stopping of the model after certain number of iterations or reducing complexity of the model, but I had no time to properly implement and explore those options.

## Conclusions

### Further project development

Given more time and resources, I would like to further develop my project in the following areas (in the order of priority):
- Investigating why my classification models performed so poorly on the test data
- More developed feature engineering including: PCA of weather variables, maybe creating a variable counting extreme weather events (e.g. exceeding two sigmas from the mean), maybe exploring aggregation by median instead of mean
- Better assignment of weather stations to BTS stations. I would like to double check my present solution and try to limit the number of BTS stations with higher (more than 50 km) distances from their weather stations. Alternatively, I would like to explore assigning maybe the weighted mean of parameters from weather stations in the neighborhood as I signalized earlier. Also, addressing the problem of weather stations in very specific locations where the weather is much different e.g. in the mountains
- Detecting outliers for the econometric and machine learning models as I unfortunately did not have time to do that
- More visualization for weather variables on the map - I believe that this is a very clear and readable method of presenting the data and I given more time I would like to make more graphs similar to the one I did for temperature
- Exploring different clustering algorithms like DBSCAN
- Completing the third machine learning task that is building a model predicting the percentage of anomalies for a given location and month :)

### Applying conclusions drawn from the data in practice

- In the process of data preprocessing, we identified several issues with the quality of the data from weather stations (missing values, incorrect values, duplicates etc.). Looking at those, we might try to implement some solutions to improve data reporting, for example better equipment, double checking entered values of parameters etc.

- Looking at the BTS station locations, some of the BTS stations were quite far away from the weather stations which may cause problems with overseeing weather conditions in those locations.

- We might want to be extra careful and do more checks for stations that are most prone to extreme weather conditions…

- … but, on the other hand, as shown later, weather conditions do not impact the percentages of anomalies in BTS stations functioning. This may actually mean that we do not have to worry about monitoring the weather conditions:) Obviously, data models are just models and we might have some reasons not to trust the data (as it was artificially generated) and use our own reasoning. 

- Clustering the weather / BTS stations allow us to identify stations often undergoing similar conditions. Therefore, if for example one station in a particular cluster starts malfunctioning, we might want to be extra careful about the other ones. 

- Similarly for classification of BTS stations that undergo a high percentage of anomalies.
